# Topics
    1. Developing a data analysis routine
    2. Reducing memory by changing data types
    3. Selecting the smallest of the largest
    4. Selecting the largest of each group by sorting
    5. Replicating nlargest with sort_values
    6. Calculating a trailing stop order price

### 1. Developing a data analysis routine

In [311]:
import pandas as pd 
import numpy as np 

In [312]:
df_college = pd.read_csv('data/college.csv')

In [313]:
df_college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [314]:
df_college.shape

(7535, 27)

In [315]:
df_college.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7535 entries, 0 to 7534
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   INSTNM              7535 non-null   object 
 1   CITY                7535 non-null   object 
 2   STABBR              7535 non-null   object 
 3   HBCU                7164 non-null   float64
 4   MENONLY             7164 non-null   float64
 5   WOMENONLY           7164 non-null   float64
 6   RELAFFIL            7535 non-null   int64  
 7   SATVRMID            1185 non-null   float64
 8   SATMTMID            1196 non-null   float64
 9   DISTANCEONLY        7164 non-null   float64
 10  UGDS                6874 non-null   float64
 11  UGDS_WHITE          6874 non-null   float64
 12  UGDS_BLACK          6874 non-null   float64
 13  UGDS_HISP           6874 non-null   float64
 14  UGDS_ASIAN          6874 non-null   float64
 15  UGDS_AIAN           6874 non-null   float64
 16  UGDS_N

In [316]:
df_college.size

203445

In [317]:
df_college.describe(include = 'object').T

,count,unique,top,freq
INSTNM,7535,7535,Everest Institute-Bensalem,1
CITY,7535,2514,New York,87
STABBR,7535,59,CA,773
MD_EARN_WNE_P10,6413,598,PrivacySuppressed,822
GRAD_DEBT_MDN_SUPP,7503,2038,PrivacySuppressed,1510


In [318]:
df_college.describe(include = [np.number]).T

,count,mean,std,min,25%,50%,75%,max
HBCU,7164.0,0.014238,0.118478,0.0,0.000000,0.00000,0.000000,1.0000
MENONLY,7164.0,0.009213,0.095546,0.0,0.000000,0.00000,0.000000,1.0000
WOMENONLY,7164.0,0.005304,0.072642,0.0,0.000000,0.00000,0.000000,1.0000
RELAFFIL,7535.0,0.190975,0.393096,0.0,0.000000,0.00000,0.000000,1.0000
SATVRMID,1185.0,522.819409,68.578862,290.0,475.000000,510.00000,555.000000,765.0000
SATMTMID,1196.0,530.765050,73.469767,310.0,482.000000,520.00000,565.000000,785.0000
DISTANCEONLY,7164.0,0.005583,0.074519,0.0,0.000000,0.00000,0.000000,1.0000
UGDS,6874.0,2356.837940,5474.275871,0.0,117.000000,412.50000,1929.500000,151558.0000
UGDS_WHITE,6874.0,0.510207,0.286958,0.0,0.267500,0.55570,0.747875,1.0000
UGDS_BLACK,6874.0,0.189997,0.224587,0.0,0.036125,0.10005,0.257700,1.0000


In [319]:
df_college.describe(include = np.number).T

,count,mean,std,min,25%,50%,75%,max
HBCU,7164.0,0.014238,0.118478,0.0,0.000000,0.00000,0.000000,1.0000
MENONLY,7164.0,0.009213,0.095546,0.0,0.000000,0.00000,0.000000,1.0000
WOMENONLY,7164.0,0.005304,0.072642,0.0,0.000000,0.00000,0.000000,1.0000
RELAFFIL,7535.0,0.190975,0.393096,0.0,0.000000,0.00000,0.000000,1.0000
SATVRMID,1185.0,522.819409,68.578862,290.0,475.000000,510.00000,555.000000,765.0000
SATMTMID,1196.0,530.765050,73.469767,310.0,482.000000,520.00000,565.000000,785.0000
DISTANCEONLY,7164.0,0.005583,0.074519,0.0,0.000000,0.00000,0.000000,1.0000
UGDS,6874.0,2356.837940,5474.275871,0.0,117.000000,412.50000,1929.500000,151558.0000
UGDS_WHITE,6874.0,0.510207,0.286958,0.0,0.267500,0.55570,0.747875,1.0000
UGDS_BLACK,6874.0,0.189997,0.224587,0.0,0.036125,0.10005,0.257700,1.0000


In [320]:
df_college.describe(include = np.object).T

,count,unique,top,freq
INSTNM,7535,7535,Everest Institute-Bensalem,1
CITY,7535,2514,New York,87
STABBR,7535,59,CA,773
MD_EARN_WNE_P10,6413,598,PrivacySuppressed,822
GRAD_DEBT_MDN_SUPP,7503,2038,PrivacySuppressed,1510


In [321]:
df_college.describe(include = [np.object, np.number, pd.Categorical]).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
INSTNM,7535,7535,Everest Institute-Bensalem,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CITY,7535,2514,New York,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STABBR,7535,59,CA,773,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HBCU,7164,NaN,NaN,NaN,0.0142379,0.118478,0,0,0,0,1
MENONLY,7164,NaN,NaN,NaN,0.00921273,0.0955465,0,0,0,0,1
WOMENONLY,7164,NaN,NaN,NaN,0.0053043,0.0726423,0,0,0,0,1
RELAFFIL,7535,NaN,NaN,NaN,0.190975,0.393096,0,0,0,0,1
SATVRMID,1185,NaN,NaN,NaN,522.819,68.5789,290,475,510,555,765
SATMTMID,1196,NaN,NaN,NaN,530.765,73.4698,310,482,520,565,785
DISTANCEONLY,7164,NaN,NaN,NaN,0.00558347,0.0745189,0,0,0,0,1


In [322]:
#Generaly the describe ethod drops the categorical values. It only renders the continous values

In [323]:
df_college.describe().T

,count,mean,std,min,25%,50%,75%,max
HBCU,7164.0,0.014238,0.118478,0.0,0.000000,0.00000,0.000000,1.0000
MENONLY,7164.0,0.009213,0.095546,0.0,0.000000,0.00000,0.000000,1.0000
WOMENONLY,7164.0,0.005304,0.072642,0.0,0.000000,0.00000,0.000000,1.0000
RELAFFIL,7535.0,0.190975,0.393096,0.0,0.000000,0.00000,0.000000,1.0000
SATVRMID,1185.0,522.819409,68.578862,290.0,475.000000,510.00000,555.000000,765.0000
SATMTMID,1196.0,530.765050,73.469767,310.0,482.000000,520.00000,565.000000,785.0000
DISTANCEONLY,7164.0,0.005583,0.074519,0.0,0.000000,0.00000,0.000000,1.0000
UGDS,6874.0,2356.837940,5474.275871,0.0,117.000000,412.50000,1929.500000,151558.0000
UGDS_WHITE,6874.0,0.510207,0.286958,0.0,0.267500,0.55570,0.747875,1.0000
UGDS_BLACK,6874.0,0.189997,0.224587,0.0,0.036125,0.10005,0.257700,1.0000


In [324]:
df_college.describe(include = np.number).T

,count,mean,std,min,25%,50%,75%,max
HBCU,7164.0,0.014238,0.118478,0.0,0.000000,0.00000,0.000000,1.0000
MENONLY,7164.0,0.009213,0.095546,0.0,0.000000,0.00000,0.000000,1.0000
WOMENONLY,7164.0,0.005304,0.072642,0.0,0.000000,0.00000,0.000000,1.0000
RELAFFIL,7535.0,0.190975,0.393096,0.0,0.000000,0.00000,0.000000,1.0000
SATVRMID,1185.0,522.819409,68.578862,290.0,475.000000,510.00000,555.000000,765.0000
SATMTMID,1196.0,530.765050,73.469767,310.0,482.000000,520.00000,565.000000,785.0000
DISTANCEONLY,7164.0,0.005583,0.074519,0.0,0.000000,0.00000,0.000000,1.0000
UGDS,6874.0,2356.837940,5474.275871,0.0,117.000000,412.50000,1929.500000,151558.0000
UGDS_WHITE,6874.0,0.510207,0.286958,0.0,0.267500,0.55570,0.747875,1.0000
UGDS_BLACK,6874.0,0.189997,0.224587,0.0,0.036125,0.10005,0.257700,1.0000


In [325]:
len(df_college.describe(include = np.number).T)

22

In [326]:
len(df_college.describe().T)

22

In [327]:
len(df_college.describe(include = 'number').T)

22

In [328]:
df_college.describe(include = 'number').T

,count,mean,std,min,25%,50%,75%,max
HBCU,7164.0,0.014238,0.118478,0.0,0.000000,0.00000,0.000000,1.0000
MENONLY,7164.0,0.009213,0.095546,0.0,0.000000,0.00000,0.000000,1.0000
WOMENONLY,7164.0,0.005304,0.072642,0.0,0.000000,0.00000,0.000000,1.0000
RELAFFIL,7535.0,0.190975,0.393096,0.0,0.000000,0.00000,0.000000,1.0000
SATVRMID,1185.0,522.819409,68.578862,290.0,475.000000,510.00000,555.000000,765.0000
SATMTMID,1196.0,530.765050,73.469767,310.0,482.000000,520.00000,565.000000,785.0000
DISTANCEONLY,7164.0,0.005583,0.074519,0.0,0.000000,0.00000,0.000000,1.0000
UGDS,6874.0,2356.837940,5474.275871,0.0,117.000000,412.50000,1929.500000,151558.0000
UGDS_WHITE,6874.0,0.510207,0.286958,0.0,0.267500,0.55570,0.747875,1.0000
UGDS_BLACK,6874.0,0.189997,0.224587,0.0,0.036125,0.10005,0.257700,1.0000


In [329]:
df_college.describe(include = [np.object, pd.Categorical]).T

,count,unique,top,freq
INSTNM,7535,7535,Everest Institute-Bensalem,1
CITY,7535,2514,New York,87
STABBR,7535,59,CA,773
MD_EARN_WNE_P10,6413,598,PrivacySuppressed,822
GRAD_DEBT_MDN_SUPP,7503,2038,PrivacySuppressed,1510


In [330]:
len(df_college.describe(include = [np.object, pd.Categorical]).T)

5

In [331]:
len(df_college.describe(include = ['number', np.object, pd.Categorical]).T)

27

In [332]:
#Schema of a DataFrame 

### 2. Reducing memory by changing data types

In [333]:
# The MEMORY usage is in BTYTES and not in bits

In [334]:
different_cols = ['RELAFFIL', 'SATMTMID', 'CURROPER', 'INSTNM', 'STABBR']

In [335]:
df_college_2 = df_college.loc[:, different_cols]
df_college_2.head()

,RELAFFIL,SATMTMID,CURROPER,INSTNM,STABBR
0,0,420.0,1,Alabama A & M University,AL
1,0,565.0,1,University of Alabama at Birmingham,AL
2,1,NaN,1,Amridge University,AL
3,0,590.0,1,University of Alabama in Huntsville,AL
4,0,430.0,1,Alabama State University,AL


In [336]:
df_college_2.dtypes

RELAFFIL      int64
SATMTMID    float64
CURROPER      int64
INSTNM       object
STABBR       object
dtype: object

In [337]:
actual_mem = df_college_2.memory_usage(deep=True)
actual_mem

Index          128
RELAFFIL     60280
SATMTMID     60280
CURROPER     60280
INSTNM      660240
STABBR      444565
dtype: int64

In [338]:
original_mem_deep_True = df_college_2.memory_usage(deep=True)
original_mem_deep_True

Index          128
RELAFFIL     60280
SATMTMID     60280
CURROPER     60280
INSTNM      660240
STABBR      444565
dtype: int64

In [339]:
original_mem_deep_False = df_college_2.memory_usage(deep=False)
original_mem_deep_False

Index         128
RELAFFIL    60280
SATMTMID    60280
CURROPER    60280
INSTNM      60280
STABBR      60280
dtype: int64

In [340]:
 df_college_2['RELAFFIL'] = df_college_2['RELAFFIL'].astype(np.int8)

In [341]:
df_college_2.dtypes

RELAFFIL       int8
SATMTMID    float64
CURROPER      int64
INSTNM       object
STABBR       object
dtype: object

In [342]:
original_mem_deep_True = df_college_2.memory_usage(deep=True)
original_mem_deep_True

Index          128
RELAFFIL      7535
SATMTMID     60280
CURROPER     60280
INSTNM      660240
STABBR      444565
dtype: int64

In [343]:
# Memory usage for RELAFFIL reduced from 60280 to 7535

In [344]:
df_college_2.select_dtypes(include=['object']).nunique()

INSTNM    7535
STABBR      59
dtype: int64

In [345]:
df_college_2['STABBR'] = df_college_2['STABBR'].astype('category')

In [346]:
df_college_2.dtypes

RELAFFIL        int8
SATMTMID     float64
CURROPER       int64
INSTNM        object
STABBR      category
dtype: object

In [347]:
original_mem_deep_True = df_college_2.memory_usage(deep=True)
original_mem_deep_True

Index          128
RELAFFIL      7535
SATMTMID     60280
CURROPER     60280
INSTNM      660699
STABBR       13576
dtype: int64

In [348]:
# Memory usage for STABBR reduced from 444565 to 13576

In [349]:
final_mem = df_college_2.memory_usage(deep=True)
final_mem

Index          128
RELAFFIL      7535
SATMTMID     60280
CURROPER     60280
INSTNM      660699
STABBR       13576
dtype: int64

In [350]:
actual_mem / final_mem

Index        1.000000
RELAFFIL     8.000000
SATMTMID     1.000000
CURROPER     1.000000
INSTNM       0.999305
STABBR      32.746391
dtype: float64

### 3. Selecting the smallest of the largest

In [351]:
df_movie = pd.read_csv('data/movie.csv')

In [352]:
df_movie.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [353]:
df_movie = df_movie[['movie_title', 'imdb_score', 'budget']]
df_movie.head()

,movie_title,imdb_score,budget
0,Avatar,7.9,237000000.0
1,Pirates of the Caribbean: At World's End,7.1,300000000.0
2,Spectre,6.8,245000000.0
3,The Dark Knight Rises,8.5,250000000.0
4,Star Wars: Episode VII - The Force Awakens,7.1,NaN


In [354]:
df_movie_2 = df_movie[['movie_title', 'imdb_score', 'budget']]

In [355]:
df_movie_2.nlargest(100, 'imdb_score').head()

,movie_title,imdb_score,budget
2725,Towering Inferno,9.5,NaN
1920,The Shawshank Redemption,9.3,25000000.0
3402,The Godfather,9.2,6000000.0
2779,Dekalog,9.1,NaN
4312,Kickboxer: Vengeance,9.1,17000000.0


In [356]:
df_movie_2.nlargest(100, 'imdb_score').nsmallest(5, 'budget')

,movie_title,imdb_score,budget
4804,Butterfly Girl,8.7,180000.0
4801,Children of Heaven,8.5,180000.0
4706,12 Angry Men,8.9,350000.0
4550,A Separation,8.4,500000.0
4636,The Other Dream Team,8.4,500000.0


### 4. Selecting the largest of each group by sorting

In [357]:
df_movie = pd.read_csv('data/movie.csv')

In [358]:
df_movie_2 = df_movie[['movie_title', 'title_year', 'imdb_score']]

In [359]:
df_movie_2.head()

,movie_title,title_year,imdb_score
0,Avatar,2009.0,7.9
1,Pirates of the Caribbean: At World's End,2007.0,7.1
2,Spectre,2015.0,6.8
3,The Dark Knight Rises,2012.0,8.5
4,Star Wars: Episode VII - The Force Awakens,NaN,7.1


In [360]:
df_movie_2.sort_values('title_year', ascending=False).head()

,movie_title,title_year,imdb_score
3884,The Veil,2016.0,4.7
2375,My Big Fat Greek Wedding 2,2016.0,6.1
2794,Miracles from Heaven,2016.0,6.8
92,Independence Day: Resurgence,2016.0,5.5
153,Kung Fu Panda 3,2016.0,7.2


In [361]:
df_movie_3 = df_movie_2.sort_values(['title_year','imdb_score'],ascending=False)

In [362]:
df_movie_3.head()

,movie_title,title_year,imdb_score
4312,Kickboxer: Vengeance,2016.0,9.1
4277,A Beginner's Guide to Snuff,2016.0,8.7
3798,Airlift,2016.0,8.5
27,Captain America: Civil War,2016.0,8.2
98,Godzilla Resurgence,2016.0,8.2


In [363]:
movie_top_year = df_movie_3.drop_duplicates(subset='title_year')

In [364]:
movie_top_year.head()

,movie_title,title_year,imdb_score
4312,Kickboxer: Vengeance,2016.0,9.1
3745,Running Forever,2015.0,8.6
4369,Queen of the Mountains,2014.0,8.7
3935,"Batman: The Dark Knight Returns, Part 2",2013.0,8.4
3,The Dark Knight Rises,2012.0,8.5


In [365]:
df_movie_4 = df_movie[['movie_title', 'title_year','content_rating', 'budget']]
df_movie_4_sorted = df_movie_4.sort_values(['title_year', 'content_rating', 'budget'], ascending=[False, False, True])
df_movie_4_sorted.drop_duplicates(subset=['title_year','content_rating']).head(10)

,movie_title,title_year,content_rating,budget
4026,Compadres,2016.0,R,3000000.0
4658,Fight to the Finish,2016.0,PG-13,150000.0
4661,Rodeo Girl,2016.0,PG,500000.0
3252,The Wailing,2016.0,Not Rated,NaN
4659,Alleluia! The Devil's Carnival,2016.0,NaN,500000.0
4731,Bizarre,2015.0,Unrated,500000.0
812,The Ridiculous 6,2015.0,TV-14,NaN
4831,The Gallows,2015.0,R,100000.0
4825,Romantic Schemer,2015.0,PG-13,125000.0
3796,R.L. Stine's Monsterville: The Cabinet of Souls,2015.0,PG,4400000.0


In [366]:
#keep = 'last'

In [367]:
df_movie_4_sorted.drop_duplicates(subset=['title_year','content_rating'], keep = 'last').head(10)

,movie_title,title_year,content_rating,budget
3312,Kicks,2016.0,R,NaN
3657,Race,2016.0,PG-13,NaN
79,The Jungle Book,2016.0,PG,175000000.0
4305,Cabin Fever,2016.0,Not Rated,NaN
4605,Irreplaceable,2016.0,NaN,NaN
4731,Bizarre,2015.0,Unrated,500000.0
812,The Ridiculous 6,2015.0,TV-14,NaN
4892,Exeter,2015.0,R,NaN
4814,Heroes of Dirt,2015.0,PG-13,NaN
3739,Christmas Eve,2015.0,PG,NaN


### 5. Replicating nlargest with sort_values

In [375]:
df_movie = pd.read_csv('data/movie.csv')
df_movie2 = df_movie[['movie_title', 'imdb_score', 'budget']]
df_movie_smallest_largest = df_movie2.nlargest(100, 'imdb_score')\
.nsmallest(5, 'budget')
df_movie_smallest_largest

,movie_title,imdb_score,budget
4804,Butterfly Girl,8.7,180000.0
4801,Children of Heaven,8.5,180000.0
4706,12 Angry Men,8.9,350000.0
4550,A Separation,8.4,500000.0
4636,The Other Dream Team,8.4,500000.0


In [372]:
df_movie2.sort_values('imdb_score', ascending=False).head(100)

,movie_title,imdb_score,budget
2725,Towering Inferno,9.5,NaN
1920,The Shawshank Redemption,9.3,25000000.0
3402,The Godfather,9.2,6000000.0
2779,Dekalog,9.1,NaN
4312,Kickboxer: Vengeance,9.1,17000000.0
...,...,...,...
3799,Anne of Green Gables,8.4,NaN
3777,Requiem for a Dream,8.4,4500000.0
3935,"Batman: The Dark Knight Returns, Part 2",8.4,3500000.0
4636,The Other Dream Team,8.4,500000.0


In [376]:
df_movie2.sort_values('imdb_score', ascending=False).head(100)\
 .sort_values('budget').head()

,movie_title,imdb_score,budget
4815,A Charlie Brown Christmas,8.4,150000.0
4801,Children of Heaven,8.5,180000.0
4804,Butterfly Girl,8.7,180000.0
4706,12 Angry Men,8.9,350000.0
4636,The Other Dream Team,8.4,500000.0


In [395]:
df_movie2.nlargest(100, 'imdb_score').tail()

,movie_title,imdb_score,budget
4023,Oldboy,8.4,3000000.0
4163,To Kill a Mockingbird,8.4,2000000.0
4395,Reservoir Dogs,8.4,1200000.0
4550,A Separation,8.4,500000.0
4636,The Other Dream Team,8.4,500000.0


In [396]:
df_movie2.sort_values('imdb_score', ascending=False) \
 .head(100).tail()

,movie_title,imdb_score,budget
3799,Anne of Green Gables,8.4,NaN
3777,Requiem for a Dream,8.4,4500000.0
3935,"Batman: The Dark Knight Returns, Part 2",8.4,3500000.0
4636,The Other Dream Team,8.4,500000.0
2455,Aliens,8.4,18500000.0


### 6. Calculating a trailing stop order price

In [398]:
import pandas_datareader as pdr
tsla = pdr.DataReader('tsla', data_source='yahoo',start='2017-1-1')
tsla.head(8)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-03,44.066002,42.192001,42.972000,43.397999,29616500.0,43.397999
2017-01-04,45.599998,42.862000,42.950001,45.397999,56067500.0,45.397999
2017-01-05,45.495998,44.389999,45.284000,45.349998,29558500.0,45.349998
2017-01-06,46.062000,45.090000,45.386002,45.801998,27639500.0,45.801998
2017-01-09,46.383999,45.599998,45.793999,46.256001,19897500.0,46.256001
2017-01-10,46.400002,45.377998,46.400002,45.973999,18300000.0,45.973999
2017-01-11,45.995998,45.335999,45.813999,45.945999,18254000.0,45.945999
2017-01-12,46.139999,45.116001,45.812000,45.917999,18951000.0,45.917999


In [401]:
tsla_close = tsla['Close']
tsla_close.head()

Date
2017-01-03    43.397999
2017-01-04    45.397999
2017-01-05    45.349998
2017-01-06    45.801998
2017-01-09    46.256001
Name: Close, dtype: float64

In [405]:
tsla_cummax = tsla_close.cummax()
tsla_cummax.head(5)

Date
2017-01-03    43.397999
2017-01-04    45.397999
2017-01-05    45.397999
2017-01-06    45.801998
2017-01-09    46.256001
Name: Close, dtype: float64

In [409]:
tsla_trailing_stop = tsla_cummax * .9
tsla_trailing_stop.head()

Date
2017-01-03    39.058199
2017-01-04    40.858199
2017-01-05    40.858199
2017-01-06    41.221798
2017-01-09    41.630400
Name: Close, dtype: float64

In [410]:
def set_trailing_loss(symbol, purchase_date, perc):
    close = pdr.DataReader(symbol, 'yahoo', start=purchase_date)['Close']
    return close.cummax() * perc

In [412]:
msft_trailing_stop = set_trailing_loss('msft', '2017-6-1', .85)
msft_trailing_stop.head()

Date
2017-06-01    59.584999
2017-06-02    60.996002
2017-06-05    61.437999
2017-06-06    61.641997
2017-06-07    61.641997
Name: Close, dtype: float64